In [18]:
!ls
!pwd

LICENSE      __pycache__  lib	      pyt_ganomaly_output
README.md    data	  options.py  requirements.txt
__init__.py  experiments  out_backup  train.py
/notebook/Masters_Project/ganomaly


Note: If running with the rtx 3070, run using the latest pytorch image from nvidia webpage

Previously used: `docker pull nvcr.io/nvidia/pytorch:21.03-py3`

In [1]:
%cd /notebook/Masters_Project/ganomaly

/notebook/Masters_Project/ganomaly


In [20]:
#!git clone https://github.com/samet-akcay/ganomaly.git

In [21]:
#!pip install -r requirements.txt

In [22]:
#!pip install mkl-fft

In [3]:
!pip install visdom
# Run via a bash terminal on the continer: python3 -m visdom.server &

In [4]:
import os
import torch
import numpy as np
import torchvision.datasets as datasets
from torchvision.datasets import MNIST
from torchvision.datasets import CIFAR10
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

from PIL import Image

In [5]:
from options import Options
from lib.data import load_data
from lib.model import Ganomaly

In [6]:
import torch

torch.cuda.is_available()

True

In [7]:
torch.__version__

'1.9.0a0+df837d0'

In [8]:
class Option:
    def __init__(self):
        # Base
        self.dataset = 'cifar10'
        self.dataroot = ''
        self.batchsize = 64
        self.workers = 8
        self.droplast = True
        self.isize = 32
        self.nc = 3
        self.nz = 100
        self.ngf = 64
        self.ndf = 64
        self.extralayers = 0
        self.device = 'gpu'
        self.gpu_ids = '0'
        self.ngpu = 1
        self.name = 'pytorch_ganomaly'
        self.model = 'ganomaly'
        self.display_server = "http://localhost"
        self.display_port = 8097
        self.display_id = 0
        self.display = True
        self.outf = './pyt_ganomaly_output'
        self.manualseed = -1
        self.abnormal_class = 'car'
        self.proportion = 0.1
        self.metric = 'roc'

        ##
        # Train
        self.print_freq = 100
        self.save_image_freq = 100
        self.save_test_images = True
        self.load_weights = False
        self.resume = ''
        self.phase = 'train'
        self.iter = 0
        self.niter = 15
        self.beta1 = 0.5
        self.lr = 0.0002
        self.w_adv = 1
        self.w_con = 50
        self.w_enc = 1
        self.isTrain = True

opt = Option()

In [9]:
def get_cifar_anomaly_dataset(trn_img, trn_lbl, tst_img, tst_lbl, abn_cls_idx=0, manualseed=-1):
    """[summary]
    Arguments:
        trn_img {np.array} -- Training images
        trn_lbl {np.array} -- Training labels
        tst_img {np.array} -- Test     images
        tst_lbl {np.array} -- Test     labels
    Keyword Arguments:
        abn_cls_idx {int} -- Anomalous class index (default: {0})
    Returns:
        [np.array] -- New training-test images and labels.
    """
    # Convert train-test labels into numpy array.
    trn_lbl = np.array(trn_lbl)
    tst_lbl = np.array(tst_lbl)

    # --
    # Find idx, img, lbl for abnormal and normal on org dataset.
    nrm_trn_idx = np.where(trn_lbl != abn_cls_idx)[0]
    abn_trn_idx = np.where(trn_lbl == abn_cls_idx)[0]
    nrm_trn_img = trn_img[nrm_trn_idx]    # Normal training images
    abn_trn_img = trn_img[abn_trn_idx]    # Abnormal training images
    nrm_trn_lbl = trn_lbl[nrm_trn_idx]    # Normal training labels
    abn_trn_lbl = trn_lbl[abn_trn_idx]    # Abnormal training labels.

    nrm_tst_idx = np.where(tst_lbl != abn_cls_idx)[0]
    abn_tst_idx = np.where(tst_lbl == abn_cls_idx)[0]
    nrm_tst_img = tst_img[nrm_tst_idx]    # Normal training images
    abn_tst_img = tst_img[abn_tst_idx]    # Abnormal training images.
    nrm_tst_lbl = tst_lbl[nrm_tst_idx]    # Normal training labels
    abn_tst_lbl = tst_lbl[abn_tst_idx]    # Abnormal training labels.

    # --
    # Assign labels to normal (0) and abnormals (1)
    nrm_trn_lbl[:] = 0
    nrm_tst_lbl[:] = 0
    abn_trn_lbl[:] = 1
    abn_tst_lbl[:] = 1

    # --
    if manualseed != -1:
        # Random seed.
        # Concatenate the original train and test sets.
        nrm_img = np.concatenate((nrm_trn_img, nrm_tst_img), axis=0)
        nrm_lbl = np.concatenate((nrm_trn_lbl, nrm_tst_lbl), axis=0)
        abn_img = np.concatenate((abn_trn_img, abn_tst_img), axis=0)
        abn_lbl = np.concatenate((abn_trn_lbl, abn_tst_lbl), axis=0)

        # Split the normal data into the new train and tests.
        idx = np.arange(len(nrm_lbl))
        np.random.seed(manualseed)
        np.random.shuffle(idx)

        nrm_trn_len = int(len(idx) * 0.80)
        nrm_trn_idx = idx[:nrm_trn_len]
        nrm_tst_idx = idx[nrm_trn_len:]

        nrm_trn_img = nrm_img[nrm_trn_idx]
        nrm_trn_lbl = nrm_lbl[nrm_trn_idx]
        nrm_tst_img = nrm_img[nrm_tst_idx]
        nrm_tst_lbl = nrm_lbl[nrm_tst_idx]

    # Create new anomaly dataset based on the following data structure:
    # - anomaly dataset
    #   . -> train
    #        . -> normal
    #   . -> test
    #        . -> normal
    #        . -> abnormal
    new_trn_img = np.copy(nrm_trn_img)
    new_trn_lbl = np.copy(nrm_trn_lbl)
    new_tst_img = np.concatenate((nrm_tst_img, abn_trn_img, abn_tst_img), axis=0)
    new_tst_lbl = np.concatenate((nrm_tst_lbl, abn_trn_lbl, abn_tst_lbl), axis=0)

    return new_trn_img, new_trn_lbl, new_tst_img, new_tst_lbl


In [10]:
splits = ['train', 'test']
drop_last_batch = {'train': True, 'test': False}
shuffle = {'train': True, 'test': False}

transform = transforms.Compose(
    [
        transforms.Resize(opt.isize),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

classes = {
    'plane': 0, 'car': 1, 'bird': 2, 'cat': 3, 'deer': 4,
    'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9
}

dataset = {}
dataset['train'] = CIFAR10(root='./data', train=True, download=True, transform=transform)
dataset['test'] = CIFAR10(root='./data', train=False, download=True, transform=transform)

dataset['train'].data, dataset['train'].targets, \
dataset['test'].data, dataset['test'].targets = get_cifar_anomaly_dataset(
    trn_img=dataset['train'].data,
    trn_lbl=dataset['train'].targets,
    tst_img=dataset['test'].data,
    tst_lbl=dataset['test'].targets,
    abn_cls_idx=classes[opt.abnormal_class],
    manualseed=opt.manualseed
)

dataloader = {x: torch.utils.data.DataLoader(dataset=dataset[x],
                                             batch_size=opt.batchsize,
                                             shuffle=shuffle[x],
                                             num_workers=int(opt.workers),
                                             drop_last=drop_last_batch[x],
                                             worker_init_fn=(None if opt.manualseed == -1
                                             else lambda x: np.random.seed(opt.manualseed)))
              for x in splits}

Files already downloaded and verified
Files already downloaded and verified


In [11]:
model = Ganomaly(opt, dataloader)
model.train()

Setting up a new session...
  0%|          | 0/703 [00:00<?, ?it/s]

>> Training model Ganomaly.


>> Training model Ganomaly. Epoch 1/15


  0%|          | 0/703 [00:00<?, ?it/s]

   Avg Run Time (ms/batch): 1.738 roc: 0.596 max roc: 0.596


...

>> Training model Ganomaly. Epoch 14/15


  0%|          | 0/703 [00:00<?, ?it/s]

   Avg Run Time (ms/batch): 1.726 roc: 0.614 max roc: 0.677


>> Training model Ganomaly. Epoch 15/15


   Avg Run Time (ms/batch): 1.843 roc: 0.620 max roc: 0.677
>> Training model Ganomaly.[Done]


In [12]:
model.test()

OrderedDict([('Avg Run Time (ms/batch)', 1.7304277420043945),
             ('roc', 0.6199634537037035)])

In [13]:
dir(model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'an_scores',
 'backward_d',
 'backward_g',
 'dataloader',
 'device',
 'epoch',
 'err_d',
 'err_d_fake',
 'err_d_real',
 'err_g',
 'err_g_adv',
 'err_g_con',
 'err_g_enc',
 'fake',
 'fake_label',
 'feat_fake',
 'feat_real',
 'fixed_input',
 'forward_d',
 'forward_g',
 'get_current_images',
 'get_errors',
 'gt',
 'gt_labels',
 'input',
 'l_adv',
 'l_bce',
 'l_con',
 'l_enc',
 'label',
 'latent_i',
 'latent_o',
 'name',
 'netd',
 'netg',
 'opt',
 'optimize_params',
 'optimizer_d',
 'optimizer_g',
 'pred_fake',
 'pred_real',
 'real_label',
 'reinit_d',
 'save_weights',
 'seed',
 'set_input',
 'test',
 'times',
 'total_steps',

In [14]:
dir(model.netd)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'apply',


In [15]:
model.visualizer.plot_performance()

TypeError: plot_performance() missing 3 required positional arguments: 'epoch', 'counter_ratio', and 'performance'

In [ ]:
#!python3 train.py --dataset cifar10  --niter 15 --abnormal_class car